**Heidelberg University**

**Data Science  Group**
    
Prof. Dr. Michael Gertz  

Ashish Chouhan, Satya Almasian, John Ziegler, Jayson Salazar, Nicolas Reuter
    
December 4, 2023
    
Natural Language Processing with Transformers

Winter Semster 2023/2024     
***

# **Assignment 3: “Transformers”**
**Due**: Monday, January 8, 2024, 2pm, via [Moodle](https://moodle.uni-heidelberg.de/course/view.php?id=19251)



### **Submission Guidelines**

- Solutions need to be uploaded as a **single** Jupyter notebook. You will find several pre-filled code segments in the notebook, your task is to fill in the missing cells.
- For the written solution, use LaTeX in markdown inside the same notebook. Do **not** hand in a separate file for it.
- Download the .zip file containing the dataset but do **not** upload it with your solution.
- It is sufficient if one person per group uploads the solution to Moodle, but make sure that the full names of all team members are given in the notebook.

***

## **Task 1: Diving into Attention** (3 + 4 + 4 + 1 = 12 points)

In this task, you work with self-attention equations and find out why multi-head attention is preferable to single-head attention.

Recall the equation of attention on slide 5-9 to compute self-attention on a series of input tokens. We simplify the formula by focusing on a single query vector $q \in R^d$, value vectors ($\{ v_1,v_2,...,v_i \},v_i \in R^d$), and key vectors ($\{ k_1,k_2,...,k_i \},k_i \in R^d$). We then have

$$
a_i=\frac{exp(q^Tk_i)}{\Sigma^n_{j=1}exp(q^Tk_j)}
$$

$$
 o= \Sigma^n_{i=1} a_i v_i
$$

with $a_i$ being the attention weight for query $q$ with respect to key $k_i$. Then the output $o$ is the new representation for the query token as a weighted average of value vectors with weights $a=\{ a_1,a_2,...,a_i \},a_i \in R^d$.
Answer the following questions with the help of the equations and the intuition behind attention that you learned in the class:



### Subtask 1: Copying  

1.   Explain why $a$ can be interpreted as a categorical distribution.
2.   This distribution is typically diffuse, where the mass is spread out between different values of $a_i$. Describe a scenario in which the categorical distribution puts all the weight on a single element, e.g., $a_j \gg \Sigma_{j\neq i}a_i$. What are the conditions on key and/or query for this to happen?
3. In this case of a single large $a$, what would the output $c$ look like and what it means intuitively?

In attention, it is easy to **copy** a value vector $v_i$ to the output $o$.





**Answer**

1. The vector $a = (a_1, ..., a_d)$ is the outcome of a softmax function applied to the input vector $(q \cdot k_1, ..., q \cdot k_d)$. Thus, $a$ has the properties of a categorical distribution, namely:
- $a_i \in [0,1]$ for each $i \in \{1, ..., d\}$.
- $\Sigma_{i=1}^da_i=1$.
Each component $a_i$ weights the importance of the corresponding value $v_i$ to the outcome $o$.

2. Consider the following scenario where the input sequence to the self-attention mechanism would be an enumeration like "Chestnut, pine, willow, oak, papaya" and one would like to compute the attention weight for "papaya". Note that all words except "papaya" represent types of trees, whereas "papaya" is only mistakenly considered a tree, so it does not fit the enumeration. Thus, the query associated with "papaya" would most probably only match the key associated with "papaya". This corresponds to the described case where $a_j \gg \Sigma_{j\neq i}a_i$, the index $j$ being the one associated with "papaya".

    The categorical distribution puts all the weight on one element typically when the input sequence contains a token (in the above case a word) which is unrelated to any other token from the input sequence. Thus, the query corresponding to this singular token matches the key associated to the same token and does not match the keys associated to the other tokens.

3. The output $c$ would be mainly influenced by the value associated to this large a, so it would almost be a copy of this value.


#### ${\color{red}{Comments\ 1.1}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
- The first answer is identical with the sample solution, by refering to the properties of softmax, which is categorical itself. (1 Point)
- The example is less mathematical than the sample solution but the example still should be correct and includes the correct properties for the key/query to focus only on one key (1 Point)
- Same answer as in the sample solution but in other words (1 Point)

✅ Point distribution ✅
3/3 Points
```


${\color{red}{⚠️Comments\ end⚠️}}$

### Subtask 2: Averaging


Instead of focusing on just one value vector $v_j$, the Transformer model can incorporate information from multiple inputs. Consider the situation where we want to incorporate information from two value vectors $v_b$ and $v_c$ with keys $k_b$ and $k_c$. In machine learning one of the ways to combine this information is through averaging of vectors $o= \frac{1}{2}(v_b+v_c)$.  It might seem hard to extract information about the original vectors $v_b$ and $v_c$ from the resulting average. But under certain conditions, one can do so. In this subtask, we look at the following cases:

1. Suppose we know the following:


* $v_b$ lies in a subspace $B$ formed by the $m$ basis vectors $\{b_1, b_2, .. , b_m\}$, while $v_c$ lies in a subspace $C$ formed by the $p$ basis vectors $\{c_1, c_2, . . . , c_p\}$ (This means that any $v_b$ and $v_c$ can be expressed as a linear combination of their basis vectors).
*   All basis vectors have the norm 1 and are orthogonal to each other.
*   The two subspaces $B$ and $C$ are orthogonal, meaning $b_j^Tc_k=0$ for all $j$ and $k$.
* Given that $\{b_1, b_2, .. , b_m\}$ are both orthogonal and form a basis for $v_b$, we know that there exists some $d_1, ..., d_m$ such that $v_b=d_1 b_1+d_2 b_2+...+d_m b_m$. Use these $d\text{s}$ to solve this task.

Using the basis vectors $\{b_1, b_2, .. , b_m\}$, construct a matrix $M$ such that for arbitrary vectors $v_b$ and $v_c$ with the given conditions, we can use $M$ to extract $v_b$ from the sum of the vector $s = v_b + v_c$. In other words, construct an $M$ such that  $ Ms = v_b$ holds.


2. If we assume that
* all key vectors are orthogonal, i.e., $k_i^Tk_j=0$ for all $i \neq j$, and
* all key vectors have the norm 1.

Find an expression for the query vector $q$ such that $o \approx \frac{1}{2}(v_b+v_c)$. Justify your answer.

**Hint:** Use your finding in subtask 1 to solve part 2.

**Hint:** If the norm of a vector $x$ is 1, then $x^Tx=1$

**Hint:** Start with writing $v_b$ and $v_c$ as the linear combination of the bases.


**Answer**

1. Let $M := A \cdot A^T$ where $A$ is a matrix with column $i$ equal to the vector $b_i$ for every $i \in \{1, ..., m\}$.

    $A =
\begin{bmatrix}
    | & | & ... & | \\
    b_1 & b_2 & ... & b_m\\
    | & | & ... & | \\
\end{bmatrix}
$
and
$M =
\begin{bmatrix}
    | & | & ... & | \\
    b_1 & b_2 & ... & b_m\\
    | & | & ... & | \\
\end{bmatrix}
\cdot
\begin{bmatrix}
    – & b_1^T & – \\
    – & b_2^T & – \\
    \vdots & \vdots & \vdots \\
    – & b_m^T & – \\
\end{bmatrix}
$

We will now show that $Ms=v_b$.

First, $Mv_c=AA^Tv_c$ and we know that $v_c = g_1c_1 + ... + g_pc_p$ for some scalars $g_1, ..., g_p$. Consider only:

$
A^Tv_c =
\begin{bmatrix}
    – & b_1^T & – \\
    – & b_2^T & – \\
    \vdots & \vdots & \vdots \\
    – & b_m^T & – \\
\end{bmatrix}
\cdot
\Sigma_{i=1}^pg_ic_i =
\Sigma_{i=1}^pg_i
\begin{bmatrix}
    – & b_1^T & – \\
    – & b_2^T & – \\
    \vdots & \vdots & \vdots \\
    – & b_m^T & – \\
\end{bmatrix}
c_i =
\Sigma_{i=1}^pg_i
\begin{bmatrix}
    b_1^T \cdot c_i\\
    b_2^T \cdot c_i\\
    \vdots \\
    b_m^T \cdot c_i\\
\end{bmatrix} = 0
$

Hence $Mv_c=0$. We show that $Mv_b=v_b$. One can easily see that:
$
A^Tv_b =
\begin{bmatrix}
    – & b_1^T & – \\
    – & b_2^T & – \\
    \vdots & \vdots & \vdots \\
    – & b_m^T & – \\
\end{bmatrix}
\cdot
\Sigma_{i=1}^md_ib_i =
\begin{bmatrix}
    d_1 \\
    d_2 \\
    \vdots \\
    d_m
\end{bmatrix}
$

Then:
$
Mv_b = AA^Tv_b =
\begin{bmatrix}
    | & | & ... & | \\
    b_1 & b_2 & ... & b_m\\
    | & | & ... & | \\
\end{bmatrix}
\cdot
\begin{bmatrix}
    d_1 \\
    d_2 \\
    \vdots \\
    d_m
\end{bmatrix} =
\Sigma_{i=1}^md_ib_i = v_b
$

2. $o \approx \frac{1}{2}(v_b+v_c)$ means that $a_b \approx \frac{1}{2}$ and  $a_c \approx \frac{1}{2}$. This would be the case if the following holds for the chosen query: $k_b^Tq \approx k_c^Tq >> k_i^Tq$ for all $i \neq b$ and $i \neq c$.

    To maximize the two dot products, let $\alpha >> 0$ and let $q = \alpha(k_b + k_c)$.

    Due to the orthogonality of the keys, note that: $k_b^Tq=k_c^Tq=\alpha, k_i^Tq=0$ for all $i \neq b$ and $i \neq c$.

    Hence, $a_b=a_c=\frac{exp(\alpha)}{n-2+2exp(\alpha)} \approx \frac{exp(\alpha)}{2exp(\alpha)} \approx \frac{1}{2}$.


#### ${\color{red}{Comments\ 1.2}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
1. The checks and general computation/proof ideas are correct, but the endresult (value for M) is incorrect since:
```
$M=B^T$ and not $M=B \cdot B^T$
```
(1.5 / 2 Points)

2.  The general idea and the computations align with the sample solution. The result is also identical, juts with other variable names. 

(2 / 2 Points)

✅ Point distribution ✅
3.5 / 4 Points


```

${\color{red}{⚠️Comments\ end⚠️}}$

### Subtask 3: Drawbacks of Single-head Attention

You might have wondered why we need multi-heads for attention. In this subtask, we look at some of the drawbacks of having a single head attention. As shown in the previous subtask, it is possible for single head attention to focus equally on two values. The same can apply to any subset of values, which therefor can become problematic.

Consider a set of key vectors $\{ k_1,k_2,...,k_n \}$, randomly sampled from a normal distribution with a known mean value of $\mu_i \in R^d$ and unknown covariance $Σ_i, i \in \{1, \ldots, n\}$, where


*   $\mu_i\text{s}$ are all orthogonal $\mu_i^T\mu_j=0$ if $i \neq j$.
*   $\mu_i\text{s}$ all have unit norm $||\mu_i||=1$.

1. For a vanishingly small $\alpha$ (not to be confused with attention weights), the covariance matrices are  $Σ_i=\alpha I, \forall i  \in \{1,2,..,n\}$, design a query $q$ in terms of the $\mu_i$ such that as before, $o= \frac{1}{2}(v_b+v_c)$ and describe why it works.

2.  Large perturbations in key value might cause problems for single head attention.  Specifically, in some cases, one key vector $k_b$ may be larger or smaller in norm than the others, while still pointing in the same direction as $\mu_b$. As an example of such a case,
consider a covariance matrix for item $b$ for vanishingly small $\alpha$ as $Σ_b=\alpha I + \frac{1}{2}(\mu_b^T\mu_b)$. This causes $k_a$ to point to roughly the same direction as $\mu_b$ but with large differences in magnitude, while for other items. Further, let $Σ_i=\alpha I\  \forall_i i \neq b$. When you sample multiple keys from the distribution $\{ k_1,k_2,...,k_n \}$ and use the $q$ vector from the pervious part, what do you expect vector $o$ to look like? Explain why this shows the drawback of single-head attention.

**Hint:**
Think about how it differs from pervious part and how $o$'s variance would be affected by the change in $Σ_b$.

**Hint:** Considering that $\mu_b^T\mu_b=1$, think of what are the ranges $Σ_b$ can take and how does that effect a sampled $k_b$ value.

**Hint:** $\frac{exp(b)}{exp(b)+exp(c)}=\frac{exp(b)}{exp(b)+exp(c)}\frac{exp(-b)}{exp(-b)}= \frac{1}{1+exp(c-b)}$

**Answer:**

1. Because $\alpha$ is vanishingly small (and thus also the variances for all $i \in \{1, ..., n\}$), we can assume $k_i \approx \mu_i$. Hence, similar to the solution in subtask 2 part 2 we can choose $q = \beta(\mu_b + \mu_c)$ for some $\beta >> 0$ large enough.

2. $k_b$ varies between $(\alpha + 0.5)\mu_b$ and $(\alpha + 1.5)\mu_b$ because $\mu_b^T\mu_b=1$. Thus: $k_b \approx \lambda\mu_b$ where $\lambda \in [0.5, 1.5]$.

    Due to $\alpha$ being vanishingly small, we can again assume that $k_i \approx \mu_i$ for all $i \neq b$. Since $\mu_i^T\mu_j=0$ for all $i \neq j$, we can assume that the dot product between $q$ and any key vector except $k_b$ and $k_c$ is $0$. For $k_b$ and $k_c$ the following happens:

    $k_b^Tq = \lambda\mu_b^T\beta(\mu_b + \mu_c)= \lambda \beta$

    $k_c^Tq = \mu_c^T\beta(\mu_b + \mu_c)= \beta$
    
    We can now compute the attention weights $a_b$ and $a_c$:

    $a_b = \frac{exp(\lambda\beta)}{n-2+exp(\lambda\beta)+exp(\beta)} \approx \frac{exp(\lambda\beta)}{exp(\lambda\beta)+exp(\beta)} \approx \frac{1}{1+exp(\beta(1-\lambda))}$

    $a_c = \frac{exp(\beta)}{n-2+exp(\lambda\beta)+exp(\beta)} \approx \frac{exp(\beta)}{exp(\lambda\beta)+exp(\beta)} \approx \frac{1}{1+exp(\beta(\lambda - 1))}$

    All other attention weights are close to zero, so we can ignore their influence on the output.

    When $\lambda = 0.5$, then $a_b \approx 0$ and $a_c \approx 1$. In this case, the output $o \approx v_c$.
    
    When $\lambda = 1.5$, then $a_b \approx 1$ and $a_c \approx 0$. Here the output is $o \approx v_b$.

    To sum up, $o$ oscillates between $v_b$ and $v_c$.


#### ${\color{red}{Comments\ 1.3}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
1. The first subtask is identical to the sample solutions explanation (1 / 1 Points)
2. The computation, the explanation and the results are the same, except some intermediate steps were omitted or not as deeply explained as in the sample solution. But in general the answer aligns with the sample solution. (3 / 3 Points)

✅ Point distribution ✅
4/ 4 Points
```


${\color{red}{⚠️Comments\ end⚠️}}$

### Subtask 4: Model Size  
1. Imagine you have an input sequence of  $l$ tokens, how much memory is required and what time complexity do we have for a single self-attention layer? (give your answer in terms of $l$)
2. If you have $N$ layers of self-attention, how  would the memory requirements and the time complexity change? (give your answer in terms of $l$ and $N$)
3. If you have $l=10,000$ and $10$ layers, with the ability to perform $10M$ operations per second, how long would it take to compute the attention output?


**Answer**

1. Given $l$ input tokens, one would first have to store $l$ query, $l$ key and $l$ value vectors. Next, $l^2$ attention weights and $l$ outputs are computed. If one stores all attention weights, then a single self-attention layer requires $O(l^2)$ memory. If one first computes all attention weigjts corresponding to one query and subsequently calculates the output, $O(l)$ memory would suffice for one self-attention layer.

    To determine the time complexity, we consider the number of necessary operations:
- $l^2$ dot products of the form $q^Tk$;
- $l^2$ exponentiations;
- $l$ sums of $l$ elements for the denominators in the formula for attention weights;
- $l^2$ divisions;
- $l^2$ scalar-vector multiplications;
- $l$ sums of $l$ elements to compute all outputs.
In total, $O(l^2)$ time is required.


2. The time complexity of $N$ layers of self-attention is: $O(N \cdot l^2)$ (each layer uses $O(l^2)$ many operations).

    If it is necessary to store the attention weights of all layers, then $O(N \cdot l^2)$ memory would be necessary.
If one does not need to keep the attention weights of all layers, then only $O(l^2)$ memory would suffice (or even $O(l)$ considering the answer at 1.).

3. If we sum up the number of operations enumerated in 1., we obtain $6l^2-2l$ many operations. By substituting l with the numerical value, we get: $599,98 \cdot 10^7$ operations. Since we can perform $10^7$ operations per second, it would take almost 600 seconds (which is 10 minutes) to compute the output.

#### ${\color{red}{Comments\ 1.4}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
1. An even deeper explanation than the sample solution was provided with the correct result at the end (0.25 / 0.25 Points)
2. Also here the explanation is deeper that the sample solution and the result is also correct
(0.25 / 0.25 Points)
3. The calculation and the result is wrong (As the sample solution states it would have been sufficent to just use the result of 1, inserting the given values and multiplying it as in solution 2 by N ) (0 / 0.5 Points)

✅ Point distribution ✅
0.5 / 1 Point


Point distribution for Task 1: 

✅ Point distribution ✅
1. 3 / 3
2. 3.5 / 4
3. 4 / 4
4. 0.5 / 1
Total: 11 / 12 Points
```


${\color{red}{⚠️Comments\ end⚠️}}$

## **Task 2: Multiple Choice Question Answering** (4 + 3 + 5 + 2 = 14 points)

In this task, you will fine-tune a transformer model on a multiple-choice task, which is the task of selecting the most plausible inputs in a given selection. The dataset used here is [SWAG](https://www.aclweb.org/anthology/D18-1009/), which is available via the Hugging Face [hub](https://huggingface.co/datasets/swag). Check the link for an overview of the dataset. SWAG is a dataset about commonsense reasoning, where each example describes a situation and then proposes four options that could apply for it.
Let's start by installing the necessary packages.

In [1]:
%pip install transformers
%pip install datasets
%pip install evaluate
%pip install accelerate -U
%pip install sentencepiece

In this task, you will use a BERT model with a `MultipleChoice` head from the Hugging Face library and then create your custom model.   Recall from the class that the BERT model has an auxiliary next sentence prediction task, in which two sentences are given to BERT separated by a `[SEP]` token and a classifier head decides if the second sentence logically follows the first one. Hugging Face has
 a `*ForMultipleChoice` architecture that uses the representation of the `[CLS]` token and a linear layer to classify if one sentence follows the other. We first start with this default architecture and then build a more complicated one in a later subtask.

### Subtask 1: Loading and Processing the Data

We use the `dataset` library to download the SWAG dataset, which already contains train, validation, and test splits.

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from datasets import load_dataset, load_metric
datasets = load_dataset("swag", "regular")
datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for swag contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/swag
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 73546
    })
    validation: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20006
    })
    test: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20005
    })
})

Lets look at the first item to see how the data looks like:

In [3]:
datasets["train"][0]

{'video-id': 'anetv_jkn6uvmqwh4',
 'fold-ind': '3416',
 'startphrase': 'Members of the procession walk down the street holding small horn brass instruments. A drum line',
 'sent1': 'Members of the procession walk down the street holding small horn brass instruments.',
 'sent2': 'A drum line',
 'gold-source': 'gold',
 'ending0': 'passes by walking down the street playing their instruments.',
 'ending1': 'has heard approaching them.',
 'ending2': "arrives and they're outside dancing and asleep.",
 'ending3': 'turns the lead singer watches the performance.',
 'label': 0}

**Question:**
Look at the dataset card on the Hugging Face hub and define what each of these fields means, with respect to the task:

*   `sent1`:
*   `sent2`:
*    `ending0`, `ending1`, `ending2` and `ending3`:
*   `label`:




**Answer**

*   `sent1`: is the first sentence
*   `sent2`: is the beginning of the second sentence
*    `ending0`, `ending1`, `ending2` and `ending3`: candidates of the endings of the second sentence, one of them is meant to be selected
*   `label`: ground truth


Write a function that displays the context and each of the four choices, following the format


```
Context:...
A-
B-
C-
D-
Ground truth: option ...
```

How you display the results is not important. You should be able to extract different parts of the data correctly and know what each field represents.

In [4]:
def explain_example(example):

  ### your code ###
  ground_truth = f"ending{example['label']}"
  log = f"""
  Context: {example['startphrase']}
  A- {example['ending0']}
  B- {example['ending1']}
  C- {example['ending2']}
  D- {example['ending3']}
  Ground truth: {ground_truth}
  """
  print(log)
  ### your code ###

In [5]:
explain_example(datasets["train"][0])


  Context: Members of the procession walk down the street holding small horn brass instruments. A drum line
  A- passes by walking down the street playing their instruments.
  B- has heard approaching them.
  C- arrives and they're outside dancing and asleep.
  D- turns the lead singer watches the performance.
  Ground truth: ending0
  


Before feeding the data into the model, we need to preprocess the text using `Tokenizer` to tokenize the inputs into tokens and put it in a format that the model expects. The tokenizer specific to the model we want to use for this task is `distilbert-base-uncased`. Complete the code below to load a fast tokenizer for this model. DistilBERT is similar to the BERT model, and we only use this particular architecture for faster training.


In [6]:
from transformers import AutoTokenizer

###your code###
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
###your code###

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
tokenizer("This is the first sentence!", "And this is the second one.")

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 999, 102, 1998, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Write a function that preprocesses the samples.
The tricky part is to put all the possible pairs of sentences in two big lists before passing them to the tokenizer.
Each **first** sentence has to be repeated 4 times to go with different ending options.
There should be a separator token between the first and second sentence, to follow the BERT input logic.
The final output is a list of 4 elements, one for each choice, where the input is transformed by the tokenizer.
For example, with a list of 2 training examples, the output includes 2 lists, where each contains 4 elements. Each of those elements is the converted input ID of the first sentence followed by the second sentence with different endings.
When calling the `tokenizer`, we use the argument `truncation=True`. This will ensure that an input longer than what the model selected can handle will be truncated to the maximum length accepted by the model.

**Hint:** Flatten the lists (all choices are flattened into a single list) before feeding them into the tokenizer and unflatten them once again for the final output.

In [8]:
### your code ###
ending_names = ["ending0", "ending1", "ending2", "ending3"]
### your code ###
def preprocess_function(examples):
  ### your code ###
    # LOG print(examples)
    # repeat each first sentence four times
    first_sentences = list(map(lambda sample: [sample for i in range(4)], examples["sent1"]))
    # second sentences possible are combination of header and ending
    question_headers = list(map(lambda sample: [sample for i in range(4)], examples["sent2"]))
    def map1(tup):
      i, sentence_list = tup
      def map2(tup2):
         j, sentence = tup2
         return sentence + " " + examples[ending_names[j]][i]
      return list(map(map2, enumerate(sentence_list)))
    second_sentences = list(map(map1, enumerate(question_headers)))

    first_sentences = [item for row in first_sentences for item in row]
    second_sentences = [item for row in second_sentences for item in row]
    SENTENCE_BREAK = " [SEP] "
    flatten = [m + SENTENCE_BREAK + n for m,n in zip(first_sentences, second_sentences)]

    tokenized = tokenizer(flatten, truncation=True)
    tokenized_dict = {k: [tokenized["input_ids"][i:i+4] for i in range(0, len(tokenized[k]), 4)] for k, v in tokenized.items()}
    # tokenized["input_ids"] = [tokenized["input_ids"][i:i+4] for i in range(0, len(tokenized["input_ids"]), 4)]
    # tokenized["attention_mask"] = [tokenized["attention_mask"][i:i+4] for i in range(0, len(tokenized["attention_mask"]), 4)]

    # LOG print(tokenized)
    # un-flatten


    return tokenized_dict
    ### your code ###

In [9]:
examples = datasets["train"][:2]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])# output should be 2 4 [30, 25, 30, 28]

2 4 [30, 25, 30, 28]


We can now apply our function to all the examples in the dataset. We use the `map` method to apply the function on all the elements of all the splits in the dataset (training, validation, and testing).
Note that we passed `batched=True` to leverage the fast tokenizer and use multi-threading to process the texts in batches concurrently.

In [10]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Our dataset is still not converted to tensors and not padded. This is the job of the `data collator`. A data collator takes a list of examples and converts them to a batch.
There is no data collator in the Hugging Face default library that works on our specific problem. We thus need to write our own one. In this collator:

*  All the inputs/attention masks are flattened.
* A flattened list is passed to the `tokenizer.pad ` method to apply dynamic padding to pad inputs to the maximum length in the batch. Output will be the size of `(batch_size * 4) x seq_length`.
* Everything needs to be unflattened for the output of the data collator.
* `input_ids` and `labels` should be returned as tensors.
* The output is a dictionary called `batch` that contains features needed for training (`input_ids`, `attention_mask`, `label`).



In [11]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class MultipleChoiceDataCollator:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        accepted_keys = ["input_ids", "attention_mask", "label"]
        if len(features[0])>len(accepted_keys):
          features=[{k: v for k, v in i.items() if k in accepted_keys} for i in features]
        ### your code ###
        #print("features")
        #print(features)
        labels = list(map(lambda sample: sample["label"], features))
        #print(labels)
        # flatten
        batch = {}
        for key in accepted_keys:
          if key == "label":
             continue
          flattened_features = list(map(lambda sample: sample[key], features))
          flattened_features = [item for row in flattened_features for item in row]
          batch[key] = flattened_features

        batch = self.tokenizer.pad(batch, padding=self.padding, max_length=self.max_length, pad_to_multiple_of=self.pad_to_multiple_of, return_tensors="pt")

        #print(batch["input_ids"].shape)
        # un-flatten
        # batch["input_ids"] = torch.tensor(batch["input_ids"]).reshape((len(features), 4, -1))
        # batch["attention_mask"] = torch.tensor(batch["attention_mask"]).reshape((len(features), 4, -1))
        batch = {k: v.reshape(len(features), len(features[0]["input_ids"]) , -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.long)

        ### your code ###
        return batch

In [12]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(2)]
batch=MultipleChoiceDataCollator(tokenizer)(features)
print(batch["input_ids"].shape)
print(batch["attention_mask"].shape)
print(batch["labels"].shape)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([2, 4, 35])
torch.Size([2, 4, 35])
torch.Size([2])


In [13]:
for i in range(4):
  print(batch["input_ids"][0][i])
  print(tokenizer.decode(batch["input_ids"][0][i]))

tensor([  101,  2372,  1997,  1996, 14385,  3328,  2091,  1996,  2395,  3173,
         2235,  7109,  8782,  5693,  1012,   102,  1037,  6943,  2240,  5235,
         2011,  3788,  2091,  1996,  2395,  2652,  2037,  5693,  1012,   102,
            0,     0,     0,     0,     0])
[CLS] members of the procession walk down the street holding small horn brass instruments. [SEP] a drum line passes by walking down the street playing their instruments. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]
tensor([  101,  2372,  1997,  1996, 14385,  3328,  2091,  1996,  2395,  3173,
         2235,  7109,  8782,  5693,  1012,   102,  1037,  6943,  2240,  2038,
         2657,  8455,  2068,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])
[CLS] members of the procession walk down the street holding small horn brass instruments. [SEP] a drum line has heard approaching them. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
tensor([  101,  2372,  1997,  1996,

#### ${\color{red}{Comments\ 2.1}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

### Subtask 2: Fine-tuning a Hugging Face Model

To fine-tune our model, we first need to download the correct architecture from Hugging Face. Import the correct class for this task and download the pre-trained checkpoint for the base class from `distilbert-base-uncased`. Note that the weights in the classification head are initialized at random.

In [14]:
### your code ###
from transformers import AutoModelForMultipleChoice
model_hf = AutoModelForMultipleChoice.from_pretrained("distilbert-base-uncased")
### your code ###

Some weights of DistilBertForMultipleChoice were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Next, we need to define our `Trainer` and pass in the correct `TrainingArguments` (a class that contains all the attributes to customize the training). Define a `TrainingArguments` that


* creates an output directory `distilbert-base-uncased-swag` to save the checkpoints and logs.
*   evaluates the model on the validation set after the `300` steps.
* a checkpoint should be saved after each `600` step and no more than 2 checkpoints should be saved in total.
* the random seed for training is `77`.
* batch size for training and evaluation: `48` (if you are running out of memory, feel free to change this setting but indicate it as a comment in your notebook, on a T4 GPU from google colab this takes about `13.2GB` of `15.0GB`).
* train for `1800` steps with a learning rate of `5e-5`, and add weight decay of `0.01` to the optimizer.
* the trainer should remove the columns from the data that are not used by the model.
* The final checkpoint should be the checkpoint that had the best overall validation metric not necessarily the last checkpoint.

**Note:** Please use GPU for to train your model. If on colab, you can use T4 GPU for free.

In [15]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="distilbert-base-uncased-swag",
                                  evaluation_strategy="steps",
                                  eval_steps=300,
                                  save_strategy="steps",
                                  save_steps=600,
                                  save_total_limit=2,
                                  seed=77,
                                  per_device_train_batch_size=48,
                                  per_device_eval_batch_size=48,
                                  learning_rate=5e-5,
                                  max_steps= 1800,
                                  weight_decay=0.01,
                                  remove_unused_columns=True,
                                  load_best_model_at_end= True)

Before we initialize the `Trainer`, we create a function that tells the trainer how to compute the metrics from the predictions. Fill the `compute_metrics` function to compute the accuracy based on the `predictions`. This object contains the prediction of the model, as well as the ground truth labels.

**Hint 1:** Keep in mind that the output of this function should be a dictionary containing the metric name and value.

**Hint 2:** Consider the shape of the example input. This is similar to the logits produced by the model.

In [16]:
import numpy as np
def compute_metrics(predictions):
  ### your code ###
   preds, label_ids = predictions
   preds = np.argmax(preds, axis=1)
   #print(preds == label_ids)
   return_dict = {}
   return_dict["accuracy"] = np.sum(preds == label_ids) / len(preds == label_ids)
  ### your code ###
   return return_dict

In [17]:
preds=np.array([[0.9,0.2,0,0],
                [0.2,0.2,0.9,0.1],
                [0.2,0.9,0,0],
                [0.2,0.1,0.8,0],
                [0.9,0.1,0.8,0],
                [0.2,1,0.4,0],
                [0.2,1,0.4,0.9],
                [1,0.1,0.4,0.3],
                [0.1,0.1,0.9,0.3],
                [0.1,0.1,0.2,1]])
label_ids=np.array([0,3,1,2,0,1,3,0,2,3])
compute_metrics((preds,label_ids))

{'accuracy': 0.8}

Now it's time to pass everything to a `Trainer` object to start the training process. Initialize a `Trainer` object and pass all the necessary information, keep in mind that we also have the optional metric computation and that we tend to run an evaluation on the validation set during training. The training should take around 30 min on Google Colab T4 GPU.

In [18]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [19]:
### your code ###
trainer = Trainer(model=model_hf.to(device),
                  args=training_args,
                  tokenizer = tokenizer,
                  data_collator=MultipleChoiceDataCollator(tokenizer),
                  train_dataset=encoded_datasets["train"],
                  eval_dataset=encoded_datasets["validation"],
                  compute_metrics=compute_metrics)
### your code ###

In [20]:
trainer.train()# should take around 30 min on Google Colab T4 GPU

Step,Training Loss,Validation Loss,Accuracy
300,No log,0.906011,0.628262
600,1.054500,0.859721,0.648655
900,1.054500,0.796303,0.674198
1200,0.929400,0.768799,0.690343
1500,0.883300,0.735419,0.705688
1800,0.883300,0.732871,0.711187


TrainOutput(global_step=1800, training_loss=0.9083089023166233, metrics={'train_runtime': 2551.0056, 'train_samples_per_second': 33.869, 'train_steps_per_second': 0.706, 'total_flos': 5678117920017504.0, 'train_loss': 0.9083089023166233, 'epoch': 1.17})

Save the model in `distilbert-base-uncased-swag/final_model`.

In [21]:
### your code ###
trainer.save_model("./distilbert-base-uncased-swag/final_model")
### your code ###

Look at the saved files and answer the following questions (it is possible to answer these questions by writing some code, but we want you to explore the saved files):

**Question:**


1.   What is the vocabulary id for the `[CLS]` and `[MASK]` tokens?
2.   What is the dropout probability for the attention layer?

**Dropout:** With dropout, certain nodes are set to the value zero in a training run, i.e. removed from the network. Thus, they have no influence on the prediction and also in the backpropagation. Thus, a new, slightly modified network architecture is built in each run and the network learns to produce good predictions without certain inputs. Read more [here](https://databasecamp.de/en/ml/dropout-layer-en).



**Answer**

1. [CLS] should be 102, [MASK] should be 104 (I analyzed according to instructions here https://huggingface.co/docs/safetensors/index)
2. Attention_dropout is 0.1 accourding to config dictionary


#### ${\color{red}{Comments\ 2.2}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

### Subtask 3: Fine-tune a Custom Model


In this case, we were lucky that Hugging Face had a pre-implemented architecture available for us to use. However, that is not always the case. Moreover, we might want to experiment beyond the default architectures to find a suitable one for a task. Therefore, it is important to learn to extend the Hugging Face models and train a custom model. The good news is that except for the model architecture the rest of the code can remain as it is.

Design a model for multiple choice model as follows:


1.   the config file for a feature extractor (must be a distilbert type) is  passed during initialization. The config file determines which model is used for feature extraction.
2.   From the `last_hidden_state` of the feature extractor, choose the `[CLS]` embedding (first one). This embedding is used as the compressed representation of first and second sentences. During pre-training it is used  for classifying whether these two sentences follow one another, making it a good candidate for our task.
3. `[CLS]` embedding is passed through a linear layer **that does not change the size of the embedding** and is passed through a tanh nonlinearity.
4. The output of tanh is passed through a dropout layer, where the dropout probability is the same as the dropout probability used for the `distilbert` model used as feature extractor.
5. The output of the previous stage is fed into another linear layer that shrinks the size of the embedding dimension to a quarter of the original size, e.g., if the embedding size is 12, the new embedding dimension is 3.
6. The output is followed by another dropout layer (you can use the one from stage 4).
7. Finally, a binary classifier is applied to determine the probability of sentence 1 being followed by sentence 2.
8. the cross-entropy loss is used to compute the loss.

**Hint:** Keep in mind that for a 4 choice system, you classify each of the four solutions independently. However, the final output should group the four logits together. For example, if input ids have the shape `[2, 4, 35]` (batch size=2, num choices=4, seq len=35), then the logits have the `[2, 4]` and labels have the dimension `[2, 1]`.



In [22]:
from transformers import DistilBertModel,BertConfig,DistilBertConfig,PretrainedConfig,PreTrainedModel,DistilBertPreTrainedModel
from torch import nn

class CustomMultipleChoice(DistilBertPreTrainedModel):
    def __init__(self, config: PretrainedConfig):
        super().__init__(config)
        ###your code ###
        self.distilbert = DistilBertModel.from_pretrained(config._name_or_path)
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()
        self.dropout = nn.Dropout(config.seq_classif_dropout)
        self.dense2 = nn.Linear(config.hidden_size, config.hidden_size // 4)
        self.classifier = nn.Linear(config.hidden_size // 4, 1)
        ###your code ###


    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
    ):
        """
        input_ids: input sentences converted to ids
        attention_mask: the attention mask
        labels:  Labels for computing the multiple choice classification loss. Indices should be in `[0, ...,num_choices-1]` where `num_choices` is the size of the second dimension of the input tensors.
        """

        num_choices = input_ids.shape[1]

        input_ids = input_ids.reshape(-1, input_ids.size(-1))
        attention_mask = attention_mask.reshape(-1, attention_mask.size(-1))
        distilbert_out = self.distilbert(input_ids = input_ids, attention_mask = attention_mask)
        last_hidden_state = distilbert_out.last_hidden_state
        cls_emb = last_hidden_state[:, 0, :]

        out = self.dropout(self.activation(self.dense(cls_emb)))
        out = self.dropout(self.dense2(out))

        # classifier and reshape
        reshaped_logits = self.classifier(out).reshape(-1, num_choices)

        loss = None
        if labels is not None:
          loss_actual = nn.CrossEntropyLoss()
          loss = loss_actual(reshaped_logits, labels.view(-1))

        ###your code ###
        return {"loss":loss,"logits":reshaped_logits}


Initialize the feature extractor with `distilbert-base-uncased` and create your custome model.

In [23]:
from transformers import AutoConfig
###your code ###
config = AutoConfig.from_pretrained("distilbert-base-uncased")
model_custom = CustomMultipleChoice(config)
###your code ###

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [24]:
for name, param in model_custom.named_parameters():
    if param.requires_grad and not name.startswith("distilbert."):
        print(name, param.data.shape)

dense.weight torch.Size([768, 768])
dense.bias torch.Size([768])
dense2.weight torch.Size([192, 768])
dense2.bias torch.Size([192])
classifier.weight torch.Size([1, 192])
classifier.bias torch.Size([1])


We keep the same training arguments but change the directory in which we save the model logs, the directory in which we save the model output and the name of the run, to `custom_model`.



In [25]:
###your code ###

training_args_custom = TrainingArguments(output_dir="custom_model",
                                  evaluation_strategy="steps",
                                  eval_steps=300,
                                  save_strategy="steps",
                                  save_steps=600,
                                  save_total_limit=2,
                                  seed=77,
                                  per_device_train_batch_size=48,
                                  per_device_eval_batch_size=48,
                                  learning_rate=5e-5,
                                  max_steps= 1800,
                                  weight_decay=0.01,
                                  remove_unused_columns=True,
                                  load_best_model_at_end= True)

###your code ###

Initialize the trainer for training the custom model.The training should take around 30 min on Google Colab T4 GPU.


In [26]:
### your code ###
trainer = Trainer(model=model_custom.to(device),
                  args=training_args_custom,
                  data_collator=MultipleChoiceDataCollator(tokenizer),
                  tokenizer = tokenizer,
                  train_dataset=encoded_datasets["train"],
                  eval_dataset=encoded_datasets["validation"],
                  compute_metrics=compute_metrics)
### your code ###
###your code ###


In [27]:
trainer.train()# should take around 30 min on Colab T4 GPU

Step,Training Loss,Validation Loss,Accuracy
300,No log,0.898943,0.630011
600,1.058800,0.852425,0.651655
900,1.058800,0.793844,0.677647
1200,0.927000,0.770358,0.690893
1500,0.878800,0.729500,0.709637
1800,0.878800,0.731600,0.711686


TrainOutput(global_step=1800, training_loss=0.9070977952745226, metrics={'train_runtime': 2558.6785, 'train_samples_per_second': 33.767, 'train_steps_per_second': 0.703, 'total_flos': 5697484563348576.0, 'train_loss': 0.9070977952745226, 'epoch': 1.17})

Save the model in `custom_model/final_model`. Note that with the custom model, you need to save it without the help of the trainer. The trainer would save the configuration but since this model is not a registered Hugging Face model only the base model would be saved. Loading the model weights is also effected by this.

In [28]:
###your code ###
model_custom.save_pretrained("./custom_model/final_model")
###your code ###

#### ${\color{red}{Comments\ 2.3}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

### Subtask 4: Evaluation and Model Comparison

Many times you do not perform the final evaluation right after training, but load the checkpoints and evaluate them on the fly. To this end, load the two models from  disk.

In [29]:
from transformers import AutoModelForMultipleChoice,AutoConfig
### your code ###
model_hf = AutoModelForMultipleChoice.from_pretrained("./distilbert-base-uncased-swag/final_model")
model_custom = CustomMultipleChoice.from_pretrained("./custom_model/final_model")
### your code ###

To evaluate the data we load the validation split using a data loader and our previously defined data collator. Note that although we had a test split we cannot use it, since there are no labels available for this split (you can check the data to confirm this).

In [30]:
from torch.utils.data import DataLoader
import evaluate

eval_dataloader = DataLoader(encoded_datasets["validation"], batch_size=64, collate_fn=MultipleChoiceDataCollator(tokenizer))

To make things easier, let's use the `evaluate` library from Hugging Face to compute the accuracy metric. Here we load `accuracy` from the `evaluate` library two times, one for the custom model and one for the Hugging Face model. Further, we put the models on eval mode. Complete the code for evaluation using the capabilities of the `evaluate` library to simultaneously compute the metric for both models.


In [31]:
from tqdm import tqdm
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
metric_dict={"custom":evaluate.load("accuracy"),"hf":evaluate.load("accuracy")} #use to compute accuracy
models_dict= {"custom":model_custom,"hf":model_hf}# use to access models

for name, model in models_dict.items():
  model.to(device)
  model.eval()

for i,batch in tqdm(enumerate(eval_dataloader), total=len(eval_dataloader)):
  ### your code ###
  #evaluate on both model on each batch
  input_ids = batch["input_ids"].to(device)
  attention_mask = batch["attention_mask"].to(device)
  labels = batch["labels"].to(device)

  #evaluate first model - Hugging Face
  with torch.no_grad():
    output_hf = model_hf(input_ids, attention_mask = attention_mask, labels = labels)
    logits_hf = output_hf.logits
  predictions_hf = torch.argmax(logits_hf, dim=1)

  #evaluate second model - custom
  with torch.no_grad():
    output_custom = model_custom(input_ids, attention_mask = attention_mask, labels = labels)
    logits_custom = output_custom["logits"]
  predictions_custom = torch.argmax(logits_custom, dim=1)

  #update metrics
  metric_dict["custom"].add_batch(predictions=predictions_custom, references = labels)
  metric_dict["hf"].add_batch(predictions=predictions_hf, references = labels)

acc_hf = metric_dict["hf"].compute()
acc_custom = metric_dict["custom"].compute()
  ### your code ###
print("Hugging Face Model :",acc_hf)
print("Custom Model :",acc_custom)

100%|██████████| 313/313 [04:48<00:00,  1.08it/s]

Hugging Face Model : {'accuracy': 0.711186644006798}
Custom Model : {'accuracy': 0.7116864940517844}


#### ${\color{red}{Comments\ 2.4}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

## **Task 3: Encoder-Decoder Architecture** (5 + 2 + 2 + 5 = 14 points)

We explored an encoder-based model (BERT) in the previous exercise. In this task, we look at another family of transformer architectures, the encoder-decoder. We use the [T5](https://arxiv.org/pdf/1910.10683.pdf) model, presented by Raffel et al.  T5 is an encoder-decoder architecture pre-trained on a multi-task mixture of unsupervised and supervised tasks. In this task, we set up a fine-tuning example for question answering using the [SQUAD](https://huggingface.co/datasets/squad) dataset. Since the actual fine-tuning is time-consuming and computational intensive for inference, we use an already pre-trained model. The main goal is to introduce you to the structure of the fine-tuning and its simplicity with the Hugging Face framework.

To fine-tune the BERT-based models, we usually add a task-specific head. On the other hand, T5 converts all NLP problems into a text-to-text format.  
It is trained using teacher forcing, meaning that we require an input sequence and a corresponding target sequence.


1.   The input sequence is fed to the model using `input_ids` from the tokenizer.
2.   The target sequence is shifted to the right, i.e., prepended by a start-sequence token and fed to the decoder using the `decoder_input_ids` (input_ids of the encoded target sequence). The target sequence is appended by EOS (end of the sentence) to denote the end of a generation and corresponds to the `labels`.
3. The task prefix defines what task is expected of T5. For example, we prepend the input sequence with `translate English to German: ` before encoding the input to tell the model to translate. T5 already has a set of pre-defined task prefixes, and it is best to stick to those since they were used during pre-training. With enough training data, you can also introduce your own custom task.


In contrast to the encoder model, where only a single `max_length` is required, for encoder-decoder architectures, one typically defines a `max_source_length` and `max_target_length`, which determine the maximum length of the input and output sequences, respectively. We must also ensure that the padding ID of the `labels` is not taken into account by the loss function. This can be done by replacing them with `-100`, which is the `ignore_index` of the `CrossEntropyLoss`.

### Subtask 1: Data Processing

We first start by loading the dataset from Hugging Face hub:

In [ ]:
from datasets import load_dataset

datasets_squad = load_dataset("squad")
datasets_squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
print("context ---->" ,datasets_squad["train"][0]["context"])
print("question ---->",datasets_squad["train"][0]["question"])
print("answers ---->",datasets_squad["train"][0]["answers"])

context ----> Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
question ----> To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
answers ----> {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


Now let's load the needed pre-trained tokenizer for `t5-small`, which is the smallest T5 model. Set the maximum sequence length to `512`.

In [ ]:
import torch
### your code ###
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
model_name = "t5-small"
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)
### your code ###

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The next step is to pre-process the dataset using the tokenizer to convert the sequences to IDs and add the special tokens.
T5 is based on the SentencePiece tokenizer, and the end of sentence token is denoted by `</s>`.
Complete the function `add_eos_to_examples` to format the input and target sequence. Your input as `input_text` should have the format `question:{question_text} context:{context_text} <EOS_Token>` and your target as `target_text` should have the format `{answer_text} <EOS_Token>`.

In [ ]:
def add_eos_to_examples(example):
    ### your code ###
    eos_token = "</s>"
    example['input_text'] = f"question:{example['question']} context:{example['context']} {eos_token}"
    example['target_text'] = f"{example['answers']} {eos_token}"
    ### your code ###
    return example

Use the `map` function to process the data, and do not set the `batched` argument.

In [ ]:
### your code ###
encoded_squad = datasets_squad.map(add_eos_to_examples)
### your code ###

In [ ]:
print("Dataset columns:",datasets_squad["train"][0].keys())
print("Encoded columns:",encoded_squad["train"].column_names)
print("Individual row columns:",encoded_squad["train"][0].keys())

Dataset columns: dict_keys(['id', 'title', 'context', 'question', 'answers'])
Encoded columns: ['id', 'title', 'context', 'question', 'answers', 'input_text', 'target_text']
Individual row columns: dict_keys(['id', 'title', 'context', 'question', 'answers', 'input_text', 'target_text'])


In [ ]:
print(encoded_squad["train"][0]["input_text"])
print(encoded_squad["train"][0]["target_text"])

question:To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? context:Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary. </s>
{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]} </s>


Complete the function `convert_to_features` that takes in the examples from the dataset and tokenizes them using the T5 tokenizer. However, our answers in this dataset are relatively short and do not require `512` tokens, in contrast to the input sequence which is a combination of question and context paragraphs and is usually long. To this end, we want to truncate the input sequence at `512` and the target sequence at `16`. If any input or target is smaller than the specified length, make sure you pad them. Finally, convert everything to PyTorch tensors to be easily used by the data collator and place them in the dictionary `encodings`.

In [ ]:
def convert_to_features(examples):
    ### your code ###

    encodings_input_text = t5_tokenizer(
        examples["input_text"],
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    encodings_target_text = t5_tokenizer(
        examples["target_text"],
        max_length=16,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    encodings = {
        'input_ids': encodings_input_text['input_ids'],
        'target_ids': encodings_target_text['input_ids'],
        'attention_mask': encodings_input_text['attention_mask'],
        'target_attention_mask': encodings_target_text['attention_mask']
    }
    ### your code ###
    return encodings

Use the `map` function to process the data.

In [ ]:
### your code ###
# encoded_squad = encoded_squad.map(convert_to_features,batched=True,batch_size=32,num_proc=2)
encoded_squad = encoded_squad.map(convert_to_features,batched=True,batch_size=32)
### your code ###

In [ ]:
print("Encoded keys:",encoded_squad["train"][0].keys())

Encoded keys: dict_keys(['id', 'title', 'context', 'question', 'answers', 'input_text', 'target_text', 'input_ids', 'target_ids', 'attention_mask', 'target_attention_mask'])


In [ ]:
encoded_squad #new columns are added

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_text', 'target_text', 'input_ids', 'target_ids', 'attention_mask', 'target_attention_mask'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_text', 'target_text', 'input_ids', 'target_ids', 'attention_mask', 'target_attention_mask'],
        num_rows: 10570
    })
})

Interestingly, although we specified PyTorch tensors as output, the type of the `input_ids` is still a list. To remedy this problem, you need to explicitly set the type of the column that contains PyTorch tensors.

In [ ]:
encoded_squad['train'][123].keys()

dict_keys(['id', 'title', 'context', 'question', 'answers', 'input_text', 'target_text', 'input_ids', 'target_ids', 'attention_mask', 'target_attention_mask'])

In [ ]:
### your code ###
print(encoded_squad["train"][0]["input_ids"])
print(encoded_squad["train"][0]["target_ids"])
print(encoded_squad["train"][0]["attention_mask"])
type(encoded_squad["train"][0]["input_ids"])

[822, 10, 3696, 4068, 410, 8, 16823, 3790, 3, 18280, 2385, 16, 507, 3449, 16, 301, 1211, 1395, 1410, 58, 2625, 10, 16768, 450, 1427, 6, 8, 496, 65, 3, 9, 6502, 1848, 5, 71, 2916, 8, 5140, 5450, 31, 7, 2045, 22161, 19, 3, 9, 7069, 12647, 13, 8, 16823, 3790, 5, 3, 29167, 16, 851, 13, 8, 5140, 5450, 11, 5008, 34, 6, 19, 3, 9, 8658, 12647, 13, 2144, 28, 6026, 3, 76, 24266, 28, 8, 9503, 96, 553, 15, 7980, 1980, 1212, 13285, 1496, 1280, 3021, 12, 8, 5140, 5450, 19, 8, 23711, 2617, 13, 8, 3, 24756, 6219, 5, 3, 29167, 1187, 8, 20605, 2617, 19, 8, 8554, 17, 235, 6, 3, 9, 17535, 286, 13, 7029, 11, 9619, 5, 94, 19, 3, 9, 16455, 13, 8, 3, 3844, 17, 235, 44, 301, 1211, 1395, 6, 1410, 213, 8, 16823, 3790, 3, 28285, 26, 120, 4283, 12, 2788, 8942, 9, 26, 1954, 264, 8371, 8283, 16, 507, 3449, 5, 486, 8, 414, 13, 8, 711, 1262, 41, 232, 16, 3, 9, 1223, 689, 24, 1979, 7, 190, 220, 12647, 7, 11, 8, 2540, 10576, 15, 201, 19, 3, 9, 650, 6, 941, 3372, 12647, 13, 3790, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

list

In [ ]:
encoded_squad.set_format(type='torch', columns=['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask'])

In [ ]:
print("Shape of the input_ids:",encoded_squad["train"]["input_ids"][0].shape)
print("Shape of the target_ids:",encoded_squad["train"]["target_ids"][0].shape)

Shape of the input_ids: torch.Size([512])
Shape of the target_ids: torch.Size([16])


The final step in the data processing is the creation of the data collator to
prepare `labels` from `target_ids` and return examples with keys as expected by the forward method of T5.
This is necessary because the trainer directly passes this dict as argument to the model so you need to check the input of T5 and rename the column based on that.
`input_ids`, `target_ids`, `attention_mask`, and `target_attention_mask` need to be stacked in a batch and the pad tokens in the target need to be set to `-100` to avoid loss computation.

In [ ]:
from dataclasses import dataclass
from transformers import DataCollator
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
from torch.nn.utils.rnn import pad_sequence
@dataclass
class T2TDataCollator:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    def __call__(self, batch):

      ### your code ###
        ## TODO: check for the right columns here

        feature_dict= {
          'input_ids': torch.stack([x['input_ids'] for x in batch]),
          'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
          'labels': torch.stack([x['target_ids'] for x in batch]),
          # 'target_attention_mask': torch.stack([x['target_attention_mask'] for x in batch])
        }

        # Assuming feature_dict is already defined
        # Replace 0 elements in each tensor with -100
        feature_dict['input_ids'][feature_dict['input_ids'] == 0] = -100
        feature_dict['attention_mask'][feature_dict['attention_mask'] == 0] = -100
        feature_dict['labels'][feature_dict['labels'] == 0] = -100
        # feature_dict['target_attention_mask'][feature_dict['target_attention_mask'] == 0] = -100



        return feature_dict
      ### your code ###


In [ ]:

encoded_squad["train"][0].keys()

dict_keys(['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask'])

In [ ]:
accepted_keys = ['input_text', 'target_text', 'input_ids', 'attention_mask', 'target_ids', 'target_attention_mask']
# features = [{k: v for k, v in encoded_squad["train"][i].items() if k in accepted_keys} for i in range(2)]
features = [{key: encoded_squad['train'][key][i] for key in accepted_keys} for i in range(2)]
batch=T2TDataCollator(t5_tokenizer)(features)
print(batch)
print(batch["input_ids"].shape)
print(batch["attention_mask"].shape)
print(batch["labels"].shape)

{'input_ids': tensor([[ 822,   10, 3696,  ..., -100, -100, -100],
        [ 822,   10, 5680,  ..., -100, -100, -100]]), 'attention_mask': tensor([[   1,    1,    1,  ..., -100, -100, -100],
        [   1,    1,    1,  ..., -100, -100, -100]]), 'labels': tensor([[    3,     2,    31,  6327,    31,    10,   784,    31, 26667,  8942,
             9,    26,  1954,   264,  8371,     1],
        [    3,     2,    31,  6327,    31,    10,   784,    31,     9,  8658,
         12647,    13,  2144,    31, 13679,     1]])}
torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([2, 16])


#### ${\color{red}{Comments\ 3.1}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

### Subtask 2: Training

For training and inference, we can use `T5ForConditionalGeneration`, which includes the language modeling head on top of the decoder. Load the `t5-small` model.

In [ ]:
### your code ###
from transformers import T5ForConditionalGeneration
t5 = T5ForConditionalGeneration.from_pretrained(model_name)
### your code ###

Next, similar to the previous task we initiate training arguments. Note that this time we are using a `Seq2SeqTrainingArguments` for a `Seq2SeqTrainer`. Set the parameters for training as follows:


*   T5 doesn't support GPU and TPU evaluation for now, so we only focus on training. You do not need to pass any parameters for evaluation setup.
*   The output directory should be named `t5-squad`.
* The T5 models need a slightly higher learning rate than the default one set in the `Trainer` when using the `AdamW` optimizer. Set the learning rate to `1e-4` and the regularization parameter to `0.01`.
* Random seed should be `77`, and we train for a maximum of `200` steps and save a checkpoint every `100` steps. A complete training of the T5 model requires far more than `200` steps, however, that is beyond the scope of this assignment.
* T5 models require a large batch size. The default model was trained with a batch size of `128`. However, we cannot fit that into a single GPU, therefore we use gradient accumulation. Set the batch size to `32` and choose the gradient accumulation step to reach the effective batch size of `128`.
* Make sure that your trainer does not remove unused columns during training, as this will cause a runtime error later on.


**Gradient accumulation:** is a technique that simulates a larger batch size by accumulating gradients from multiple small batches before performing a weight update.



In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="t5-squad",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,  # Effective batch size = 32 * 4 = 128
    evaluation_strategy="steps",  # No evaluation during training
    save_total_limit=2,  # Only keep the last 2 checkpoints
    learning_rate=1e-4,
    weight_decay=0.01,
    num_train_epochs=2,  # Adjust the number of epochs as needed
    save_steps=100,  # Save a checkpoint every 100 steps
    logging_dir="logs",
    logging_steps=10,
    seed=77,
    remove_unused_columns=False,  # Do not remove unused columns
)

Once again make sure that you are using GPU before running the cell below.
Initilize your `Seq2SeqTrainer` with inputs necessary for training. The training should take around 15 min on Google Colab T4 GPU.


##### NOTE: Because of some importing and exporting issues, I did not upload this to Google Colab for training, but feel free to try to see that it works.

In [ ]:
# Initialize our Trainer
from transformers import Seq2SeqTrainer
# Initialize the trainer

data_collator = T2TDataCollator(t5_tokenizer)


trainer = Seq2SeqTrainer(
    model=t5,
    args=training_args,
    data_collator=data_collator,
    train_dataset=encoded_squad["train"]
)

# Start training
trainer.train()

  0%|          | 0/1368 [00:00<?, ?it/s]

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
# trainer.train()

#### ${\color{red}{Comments\ 3.2}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

### Subtask 3: Inference

Our trained model has seen far too few instances to make a coherent prediction. To this end, we load an already trained checkpoint from Hugging Face and perform inference. Load this [model](https://huggingface.co/mrm8488/t5-base-finetuned-squadv2) and the respective tokenizer. Note that we are loading a `base` model that is slightly larger than `t5-small`.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
### your code ###
t5_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-squadv2")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-squadv2")
### your code ###

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can 

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

c:\Users\priot\anaconda3\envs\computing\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\priot\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


At inference time for T5, it is recommended to use the `generate()` function. This auto-regressively generates the decoder output. Complete the code for the `get_answer` function, which gives a model, a tokenizer, and a question and context pair, and generates the answer from the context given. The output should be the answer to the given question in natural text (without the special tokens).

**Hint:** Many of the steps are similar to how you prepared your input data for the model.

In [ ]:
def get_answer(tokenizer,model, question, context):
  ### your code ###
  eos_token = "</s>"
  input_text =f"question:{question} context:{context} {eos_token}"

  print(input_text)

  features = tokenizer(
        input_text,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )


  answer= model.generate(input_ids=features['input_ids'],
               attention_mask=features['attention_mask'])

  answer_text = tokenizer.decode(answer[0], skip_special_tokens=True)

  ### your code ###
  return answer_text

Let's try it with an example.

In [ ]:
context = "Sarah has joined NLP for transformers class and is working on her research project with the support of Harry."
question = "Who is supporting Sarah?"

get_answer(t5_tokenizer,t5_model,question, context)###your answer should be "Harry"

question:Who is supporting Sarah? context:Sarah has joined NLP for transformers class and is working on her research project with the support of Harry. </s>


c:\Users\priot\anaconda3\envs\computing\Lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Harry'

In [ ]:
context = "TPUs are more power efficient in comparison to GPUs making them a better choice for machine learning projects."
question = "What is better for machine learning projects?"

get_answer(t5_tokenizer,t5_model,question, context)###your answer should be "TPUs"

question:What is better for machine learning projects? context:TPUs are more power efficient in comparison to GPUs making them a better choice for machine learning projects. </s>


'GPUs'

#### ${\color{red}{Comments\ 3.3}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$

### Subtask 4: T5 Paper

To answer questions of the final subtask you need to have a general overview of the [T5 paper](https://arxiv.org/pdf/1910.10683.pdf).



1.   Describe what a “text-to-text format" is and how T5 processes input and output for text classification tasks? What are the possible complications with a predefined set of classes?
2.   Describe the "masked language modeling" and "word dropout" unsupervised objective with sentinel tokens. Give an example of how this would look in a single sentence.
3. Explain "fully-visible", "causal" and "causal masking with prefix" masking.
4. Briefly describe "adapter layers" and "gradual unfreezing" as methods for fine-tuning on fewer parameters.



**Answer**


1. Special tags are added to the input text (such as translate, cola, stsb, summarize) and the output is expected to contain the corresponding outcome after training. Complications may arise when trying to expand the capabilities of the model (i.e. addding more classes) or when classes overlap or there is a class imbalance in the training set.

2. The masked language modeling, inspired by BERT, consists in masking certain tokens from the input sequence with a sentinel token (consecutive tokens are masked with a single sentinel token; this is done only as a way to reduce the computational cost of the pre-training) and then predicting them as outputs. The "word dropout" objective (Bowman et al., 2015) consists in randomly sampling and dropping 15% of the tokens in the input sequence.

    Example (from paper):

    Original Text:
    Thank you for inviting me to your party last week.

    Inputs (masking):
    Thank you <X> me to your party <Y> week.

    Targests:
    <X> for inviting <Y> last <Z>


3. Fully-visible masking: used for the encoder; self-attention attends to any entry of the input when producing its output.
   Causal masking: used for the decoder; this type of masking prevents the model from "seeing into the future" i.e for producing the ith output, the model does not look at the jth input where j > i.
   Causal masking with prefix: allows the self-attention mechanism to use fully-visible masking on a portoin of the input sequence (soft combination between fully visible and causal)


4. Adaptive layers: after the preexisting feed-forward networks in the Tranformer architecture, add blocks of Linear(d,h) - Relu - Linear(h,d). Only these parameters are updated during fine-tuning.
   
   Gradual Unfreezing: starting from the top of the encoder and decoder (in parallel), gradually unfreeze layers during fine-tuning and move deeper in the network.


#### ${\color{red}{Comments\ 3.4}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
cross-feedback comment section
```


${\color{red}{⚠️Comments\ end⚠️}}$